## Let's Import all the basic Libraries 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### importing the data from trusted site like yahoo finance  

In [7]:
import yfinance as stocks_data
ticker = input("Inter the Symbol: ")
data = stocks_data.Ticker(ticker)
df = data.history(period="MAX")
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1996-01-01 00:00:00+05:30,5.301994,5.321977,5.236719,5.308655,10242229,0.0,0.0
1996-01-02 00:00:00+05:30,5.301994,5.321976,5.196753,5.210074,16954313,0.0,0.0
1996-01-03 00:00:00+05:30,5.210074,5.328637,5.196753,5.211407,13514114,0.0,0.0
1996-01-04 00:00:00+05:30,5.128814,5.128814,4.928989,5.046220,34785820,0.0,0.0
1996-01-05 00:00:00+05:30,4.984941,5.008919,4.895686,4.946308,30138033,0.0,0.0
...,...,...,...,...,...,...,...
2024-02-27 00:00:00+05:30,142.649994,144.500000,142.000000,144.199997,24119469,0.0,0.0
2024-02-28 00:00:00+05:30,144.050003,144.949997,140.050003,140.750000,20721926,0.0,0.0
2024-02-29 00:00:00+05:30,140.800003,141.850006,139.399994,140.850006,32248156,0.0,0.0
